In [1]:
from keras.models import Sequential
from keras.layers import LSTM, TimeDistributed, Dense, RepeatVector, Masking
import numpy
import os
import re
import pandas
import sklearn.preprocessing

Using TensorFlow backend.


In [2]:
csvs = list(filter(lambda file: re.match(r'^\d{8} Test Data\.txt$', file) is not None, os.listdir()))

In [4]:
dfs = []

for csv in csvs:
    
    df = pandas.read_csv('./' + csv, sep='\t', header=0)
    df = df[['Pedal_accel_pos_CAN[per]', 'Dyno_Spd[mph]']]
    dfs.append(df)

In [5]:
MAXLEN = max([len(df) for df in dfs])

In [6]:
X = numpy.full([len(dfs), MAXLEN, 1], -1.0)
Y = numpy.full([len(dfs), MAXLEN, 1], -1.0)

min_max_scaler = sklearn.preprocessing.MinMaxScaler()

for i in range(len(dfs)):
    X[i][:len(dfs[i])] += min_max_scaler.fit_transform(dfs[i]['Pedal_accel_pos_CAN[per]'].values.reshape(-1, 1)) + 1
    Y[i][:len(dfs[i])] += numpy.array([dfs[i]['Dyno_Spd[mph]']]).T + 1

In [7]:
indices = numpy.arange(len(X))

X = X[indices]
Y = Y[indices]

In [8]:
split = len(X) - len(X) // 10
X_train, X_val = X[:split], X[split:]
Y_train, Y_val = Y[:split], Y[split:]

In [9]:
HIDDEN_SIZE = 10
BATCH_SIZE = 1
LAYERS = 1

In [10]:
model = Sequential()
model.add(Masking(-1, input_shape=(MAXLEN, 1)))
model.add(LSTM(HIDDEN_SIZE))
model.add(RepeatVector(MAXLEN))
for _ in range(LAYERS):
    model.add(LSTM(HIDDEN_SIZE, return_sequences=True))
model.add(TimeDistributed(Dense(1, activation='linear')))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_1 (Masking)          (None, 61561, 1)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 10)                480       
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 61561, 10)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 61561, 10)         840       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 61561, 1)          11        
Total params: 1,331
Trainable params: 1,331
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=1, validation_data=(X_val, Y_val))

Train on 86 samples, validate on 9 samples
Epoch 1/1
